<a href="https://colab.research.google.com/github/iabhay/Audio-Sentimental-Analysis/blob/main/Sentimental_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from time import sleep
import pandas as pd

In [ ]:
# Obtained in Step 2
API_key = "535b2431381742b297a92085de5c951c"

## Location on local machine
file = '/datasets/conversation/Power_English_Update.mp3'

In [ ]:
headers = {
    'authorization': API_key, 
    'content-type': 'application/json',
}

endpoint = 'https://api.assemblyai.com/v2/upload'

In [ ]:
def read_file(file):

    with open(file, 'rb') as f:
        while True:
            data = f.read(5_242_880)
            if not data:
                break
            yield data

In [ ]:
res_upload = requests.post(
    endpoint, 
    headers=headers, 
    data=read_file(file)
)

print(res_upload.json())

upload_url = res_upload.json().get('upload_url')

{'upload_url': 'https://cdn.assemblyai.com/upload/c63833da-148f-494c-8318-e5eabb7d4eef'}


# New Section

In [ ]:
endpoint = "https://api.assemblyai.com/v2/transcript"

json = {
    "audio_url": upload_url,
    "sentiment_analysis": True,
    "speaker_labels": True
}

headers = {
    "authorization": API_key,
    "content-type": "application/json"
}

response = requests.post(endpoint, json=json, headers=headers)

response.json()

{'id': 'r7yvxbpnwy-64c8-47e5-aac7-ff935d16ff08',
 'language_model': 'assemblyai_default',
 'acoustic_model': 'assemblyai_default',
 'language_code': 'en_us',
 'status': 'queued',
 'audio_url': 'https://cdn.assemblyai.com/upload/c63833da-148f-494c-8318-e5eabb7d4eef',
 'text': None,
 'words': None,
 'utterances': None,
 'confidence': None,
 'audio_duration': None,
 'punctuate': True,
 'format_text': True,
 'dual_channel': None,
 'webhook_url': None,
 'webhook_status_code': None,
 'webhook_auth': False,
 'webhook_auth_header_name': None,
 'speed_boost': False,
 'auto_highlights_result': None,
 'auto_highlights': False,
 'audio_start_from': None,
 'audio_end_at': None,
 'word_boost': [],
 'boost_param': None,
 'filter_profanity': False,
 'redact_pii': False,
 'redact_pii_audio': False,
 'redact_pii_audio_quality': None,
 'redact_pii_policies': None,
 'redact_pii_sub': None,
 'speaker_labels': True,
 'content_safety': False,
 'iab_categories': False,
 'content_safety_labels': {},
 'iab_cate

In [ ]:
response_id = response.json()['id']

endpoint = f"https://api.assemblyai.com/v2/transcript/{response_id}"

headers = {
    "authorization": API_key,
}
response = requests.get(endpoint, headers=headers)

response.json()

{'id': 'r7yvxbpnwy-64c8-47e5-aac7-ff935d16ff08',
 'language_model': 'assemblyai_default',
 'acoustic_model': 'assemblyai_default',
 'language_code': 'en_us',
 'status': 'processing',
 'audio_url': 'https://cdn.assemblyai.com/upload/c63833da-148f-494c-8318-e5eabb7d4eef',
 'text': None,
 'words': None,
 'utterances': None,
 'confidence': None,
 'audio_duration': None,
 'punctuate': True,
 'format_text': True,
 'dual_channel': None,
 'webhook_url': None,
 'webhook_status_code': None,
 'webhook_auth': False,
 'webhook_auth_header_name': None,
 'speed_boost': False,
 'auto_highlights_result': None,
 'auto_highlights': False,
 'audio_start_from': None,
 'audio_end_at': None,
 'word_boost': [],
 'boost_param': None,
 'filter_profanity': False,
 'redact_pii': False,
 'redact_pii_audio': False,
 'redact_pii_audio_quality': None,
 'redact_pii_policies': None,
 'redact_pii_sub': None,
 'speaker_labels': True,
 'content_safety': False,
 'iab_categories': False,
 'content_safety_labels': {},
 'iab_

In [ ]:
current_status = "queued"
response_id = response.json()['id']
endpoint = f"https://api.assemblyai.com/v2/transcript/{response_id}"
headers = {
    "authorization": API_key,
}

while current_status not in ("completed", "error"):
    
    response = requests.get(endpoint, headers=headers)
    current_status = response.json()['status']
    
    if current_status in ("completed", "error"):
        print(response)
    else:
        sleep(10)
        
current_status

<Response [200]>


'completed'

In [ ]:
response.json()

{'id': 'r7yvxbpnwy-64c8-47e5-aac7-ff935d16ff08',
 'language_model': 'assemblyai_default',
 'acoustic_model': 'assemblyai_default',
 'language_code': 'en_us',
 'status': 'completed',
 'audio_url': 'https://cdn.assemblyai.com/upload/c63833da-148f-494c-8318-e5eabb7d4eef',
 'text': "Hi. This is AJ with another effortless English podcast. So today is Monday and I'm here in San Francisco. I'm back in San Francisco. Last week we were in Texas. We were at a teaching conference, and the leader of of the teaching conference was Blaine Ray. And first, I just want to say some nice things about Blaine Ray because he's a super nice person. He's a fantastic teacher. He's a great teacher trainer. He really helps teachers become better and better. And I went, Tomoe went, and Kristen went and Joe went. We all learned a lot. We will all improve our teaching as a result of going to the conference. So. Thank you, Blaine. Ray. You are great. See, the next thing that happened last week was that our computers

In [ ]:
response.json()["sentiment_analysis_results"][1]

{'text': 'This is AJ with another effortless English podcast.',
 'start': 2980,
 'end': 8190,
 'sentiment': 'POSITIVE',
 'confidence': 0.7118508815765381,
 'speaker': 'A'}

In [ ]:
sent_data = []

for idx, sentence in enumerate(response.json()["sentiment_analysis_results"]):

    sent = sentence["text"]
    sentiment = sentence["sentiment"]
    duration = (sentence["end"] - sentence["start"]) / 1000
    speaker = sentence["speaker"]
    sent_data.append([idx +1 , sent, duration, speaker, sentiment])

sent_data = pd.DataFrame(sent_data, 
                         columns = ["SentenceID", "Text", "Duration", 
                                    "Speaker", "Sentiment"])

In [ ]:
sent_data.Speaker.value_counts()

A    122
Name: Speaker, dtype: int64

In [ ]:
100*sent_data.Speaker.value_counts(normalize=True)

A    100.0
Name: Speaker, dtype: float64

In [ ]:
sent_data.groupby("Speaker").Duration.sum()

Speaker
A    429.766
Name: Duration, dtype: float64

In [ ]:
sent_data.Sentiment.value_counts()

NEUTRAL     69
POSITIVE    35
NEGATIVE    18
Name: Sentiment, dtype: int64

In [ ]:
100*sent_data.Sentiment.value_counts(normalize=True)

NEUTRAL     56.557377
POSITIVE    28.688525
NEGATIVE    14.754098
Name: Sentiment, dtype: float64

In [ ]:
print(pd.crosstab(sent_data.Speaker, sent_data.Sentiment))

Sentiment  NEGATIVE  NEUTRAL  POSITIVE
Speaker                               
A                18       69        35


In [ ]:
sent_data.groupby("Sentiment").Duration.mean()

Sentiment
NEGATIVE    4.060556
NEUTRAL     3.850464
POSITIVE    2.599829
Name: Duration, dtype: float64